In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

PATH_TO_DATA = '../data/'
PATH_TO_OUTPUT_DATA = '../output-data/'

In [22]:
# concat all rows 
i = '2000ANA.EVA'
results = {}
game_id = ''
with open(PATH_TO_DATA + 'event/' + i, 'r') as f:
    lines = f.readlines()

In [178]:
# takes an action and sees how many runs scored
def compute_num_runs(running):
    runs = 0
    if ('3-H' in running): 
        # +1 RS 
        runs = runs + 1
    if ('2-H' in running): 
        # +1 RS 
        runs = runs + 1
    if ('1-H' in running): 
        # +1 RS 
        runs = runs + 1
    return runs

In [289]:
rows = []

for line in lines: 
    if (line[0:2] == 'id'):
        game_date = line.split(',')[1][3:-1]
        game_id = game_date[-1:]
        game_date = game_date[:-1]
    
    # separate info
    if (line[0:4] == 'info'):
        if (line.split(',')[1] == 'visteam'):
            away_team = line.split(',')[2][:-1]
        if (line.split(',')[1] == 'hometeam'):
            home_team = line.split(',')[2][:-1]
    if (line[0:4] == 'play'):
        row = [game_date, game_id]
        play_data = line.split(',')
        # not important to keep inning
        player_team = home_team if play_data[2] == '1' else away_team
        row.append(player_team)
        row.append(play_data[3])
        
#         'AB', 'RS', 'H', 'HR', 'TB', 'BB', 'SF'
        # record play data
        next_play = [0, 0, 0, 0, 0, 0, 0]
        play_details = play_data[6][:-1]

        ##### CASE 1: WHEN NO DIRECTION IS INDICATED
        if (len(play_details.split('/')) == 1):
            # no fielding occured
            if (len(play_details.split('.')) > 1):
                ##### CASE 1: BATTERS ON BASE AND CAN SCORE
                action = play_details.split('.')
                # we ignore walk + events
                if ((action[0] == 'W') or (action[0] == 'HP') 
                    or (action[0] == 'IW') or (action[0] == 'I')):
                    # BB
                    next_play[1] = compute_num_runs(action[1])
                    next_play[5] = 1
                    rows.append(row + next_play) # ADD TO TABLE
                elif (((action[0][0] == 'S') and ('SB' not in action[0]))
                      or ((action[0][0] == 'D') and ('DI' not in action[0]))
                      or (action[0][0] == 'T')):
                    # single, double or triple
                    next_play[0] = 1
                    next_play[1] = compute_num_runs(action[1])
                    next_play[2] = 1
                    next_play[4] = 1 if (action[0][0] == 'S') else (2 if (action[0][0] == 'D') else 3)
                    rows.append(row + next_play) # ADD TO TABLE
                else:
                    # 1 AB no result
                    if ((action[0][0:2] != 'SB') and (action[0][0:2] != 'CS') 
                        and (action[0][0:2] != 'WP') and (action[0][0:2] != 'NP')):
                        next_play[0] = 1
                    next_play[1] = compute_num_runs(action[1])
                    rows.append(row + next_play) # ADD TO TABLE
            else:
                ##### CASE 1B: NO BATTER MOVEMENT; NOBODY ON BASE
                if ((play_details == 'W') or (play_details == 'HP') 
                    or (play_details == 'IW') or (play_details == 'I')):
                    # BB
                    next_play[1] = 0 
                    next_play[5] = 1
                    rows.append(row + next_play) # ADD TO TABLE
                elif (((play_details[0] == 'S')and ('SB' not in play_details))
                      or ((play_details[0] == 'D') and ('DI' not in play_details)) 
                      or (play_details[0] == 'T')):
                    # single, double or triple
                    next_play[0] = 1
                    next_play[1] = 0 # no runs scored
                    next_play[2] = 1
                    next_play[4] = 1 if (play_details[0] == 'S') else (2 if (play_details[0] == 'D') else 3)
                    rows.append(row + next_play) # ADD TO TABLE
                else:
                    # 1 AB no result
                    if ((play_details[0:2] != 'SB') and (play_details[0:2] != 'CS') 
                        and (play_details[0:2] != 'WP') and (play_details[0:2] != 'NP')):
                        next_play[0] = 1
                    rows.append(row + next_play) # ADD TO TABLE
        
        ##### CASE 2: WHEN DIRECTION IS INDICATED
        else:
            play_detail_2 = play_details.split('/')
            if (len(play_detail_2[1].split('.')) > 1):
                ##### CASE 2a: BATTERS ON BASE AND CAN SCORE
                action = play_detail_2[0]
                runners = play_detail_2[1].split('.')[1]
                if ((action == 'H') or (action == 'H9') or (action == 'HR') or (action == 'HR9')):
                    # home runs aren't counted
                    next_play[0] = 1
                    next_play[1] = compute_num_runs(runners) + 1 # home run implicit
                    next_play[2] = 1
                    next_play[3] = 1
                    next_play[4] = 4
                    rows.append(row + next_play) # ADD TO TABLE
                elif (((action[0] == 'S') and ('SB' not in action))
                      or ((action[0] == 'D') and ('DI' not in action)) 
                      or (action[0] == 'T')):
                    # single, double or triple
                    next_play[0] = 1
                    next_play[1] = compute_num_runs(runners)
                    next_play[2] = 1
                    next_play[4] = 1 if (action[0] == 'S') else (2 if (action[0] == 'D') else 3)
                    rows.append(row + next_play) # ADD TO TABLE
                elif(play_detail_2[1].split('.')[0] == 'SF'):
                    # sacrifice fly
                    next_play[1] = 1
                    next_play[6] = 1
                    rows.append(row + next_play) # ADD TO TABLE
                elif(play_detail_2[1].split('.')[0] == 'SH'):
                    # sacrifice hit; not counted as AB or OBP though
                    next_play[1] = compute_num_runs(runners)
                    rows.append(row + next_play) # ADD TO TABLE
                else:
                    # 1 AB no result
                    if ((action[0:2] != 'SB') and (action[0:2] != 'CS') and (action[0:2] != 'WP')):
                        next_play[0] = 1
                    next_play[1] = compute_num_runs(runners)
                    rows.append(row + next_play) # ADD TO TABLE                
            else:
                ##### CASE 2b: BATTERS NOT ON BASE, NO SAC FLIES!
                action = play_detail_2[0]
                if (((action[0] == 'S') and ('SB' not in action))
                    or ((action[0] == 'D') and ('DI' not in action)) 
                    or (action[0] == 'T')):
                    # single, double or triple
                    next_play[0] = 1
                    next_play[1] = 0 # no runs scored
                    next_play[2] = 1
                    next_play[4] = 1 if (action[0] == 'S') else (2 if (action[0] == 'D') else 3)
                    rows.append(row + next_play) # ADD TO TABLE
                elif ((action == 'H') or (action == 'H9') or (action == 'HR') or (action == 'HR9')):
                    # home runs aren't counted
                    next_play[0] = 1
                    next_play[1] = 1 # home run implicit
                    next_play[2] = 1
                    next_play[3] = 1
                    next_play[4] = 4
                    rows.append(row + next_play) # ADD TO TABLE
                else:
                    # 1 AB no result
                    if ((action[0:2] != 'SB') and (action[0:2] != 'CS') and (action[0:2] != 'WP')):
                        next_play[0] = 1
                    rows.append(row + next_play) # ADD TO TABLE  

['S8', '1-2']
['S8', '2-H;1-2']
['S9', '1-3']
['S8', '1-2']
['S8', '2-H;1-2']
['S8', '3-H(UR);2-H(UR)']
['S8', '1-H(UR)']
['D7', '2-H;1-3']
['S8', '1-2']
['D9', '2-H;1-H']
['S7', '2-H;1-2']
['S7', '3-H;1-2']
['S9', '2-H;BX1(934)']
['S7', '2-3']
['S8', '3-H;2-H']
['D9', '1-H']
['S8', '2-H']
['S', '2-3']
['S7', '1-2']
['S', '2-H']
['S8', '2-H;1-2']
['S', '2-3;1-2']
['D9', '3-H']
['S7', '2-H']
['S8', '2-H;B-2(THH)']
['D9', '1-3']
['S', '2-H']
['S9', '1-3']
['S8', '2-3']
['S8', '3-H']
['S9', '1X2(96)']
['D9', '2-H']
['S7', '3-H;1-2']
['S7', '1-2']
['S8', '1-2']
['S7', '1-3']
['S9', '3-H(UR)']
['S8', '1-3']
['D7', '1-3']
['S7', '3-H;2-H(UR)']
['S8', '1-2']
['S9', '1-3']
['S8', '2-3;1-2']
['S9', '2-3']
['S8', '3-H;1-2']
['S8', '1-2']
['S9', '2-H(UR);1-3']
['S9', '1-2']
['D9', '2-H']
['S', '1-2']
['S8', '3-H;2-H;1-2']
['S7', '3-H;1-2']
['S', '1-2']
['D9', '1-H']
['D9', '1-H']
['S7', '1-2']
['D9', '3-H;1-3']
['S9', '3-H;2-3']
['S7', '3-H;1-2']
['S', '1-2']
['D7', '1-3']
['D9', '1-3']
['D9', '1

In [290]:
## BB = BB + HBP
player_df = pd.DataFrame(rows, 
        columns = ['date', 'game_id', 'team', 'player_id', 'AB', 'RS', 'H', 'HR', 'TB', 'BB', 'SF'])
player_df

,date,game_id,team,player_id,AB,RS,H,HR,TB,BB,SF
0,20000403,0,NYA,knobc001,1,0,1,0,1,0,0
1,20000403,0,NYA,jeted001,0,0,0,0,0,0,0
2,20000403,0,NYA,jeted001,1,0,0,0,0,0,0
3,20000403,0,NYA,oneip001,0,0,0,0,0,1,0
4,20000403,0,NYA,willb002,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7279,20001001,0,ANA,gil-b001,0,0,0,0,0,0,0
7280,20001001,0,ANA,palmo001,0,0,0,0,0,1,0
7281,20001001,0,ANA,erstd001,1,0,0,0,0,0,0
7282,20001001,0,ANA,erstd001,1,0,0,0,0,0,0


In [291]:
player_df.groupby('team').sum()

,AB,RS,H,HR,TB,BB,SF
team,,,,,,,
ANA,2767,447,788,130,1346,339,23
BAL,204,26,52,5,77,26,3
BOS,190,19,37,8,68,22,1
CHA,167,27,58,4,84,18,0
CLE,106,19,30,4,51,15,2
COL,103,10,26,1,38,7,1
DET,207,23,57,8,93,18,2
KCA,216,34,61,5,82,22,5
LAN,107,20,31,7,58,15,1


In [93]:
print(play_details)

6/P6

